In [ ]:
import tensorflow as tf
import api.capital_features_api as cf
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import numpy as np

tf.config.list_physical_devices(device_type='GPU')

In [ ]:
train_data = cf.get_train_data(cf.prod_train_data_url, train_config_id=35602)
validation_data = cf.get_train_data(cf.prod_train_data_url, train_config_id=35603)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data['featuresT'], train_data['upSignals']))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_data['featuresT'], validation_data['upSignals']))
BATCH_SIZE = 64  # 你可以根据需要调整这个值
train_dataset = train_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

In [ ]:
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data['upSignals']),
    y=train_data['upSignals']
)
class_weights = dict(zip(np.unique(train_data['upSignals']), weights))

In [ ]:
model = Sequential([
    InputLayer(input_shape=(6, 11)),
    LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    BatchNormalization(),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    BatchNormalization(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1, activation='sigmoid'),
])
model.summary()
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='acc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-6)

In [ ]:
his_up = model.fit(train_dataset, epochs=20, validation_data=validation_dataset, verbose=1,
                   callbacks=[early_stopping, reduce_lr, lr_scheduler], class_weight=class_weights)

In [ ]:
plt.plot(his_up.history['acc'], label='Training_UP')
plt.plot(his_up.history['val_acc'], label='Validation_UP')
plt.legend()
plt.show()